This notebook was inpired by [this LlamaIndex notebook](https://colab.research.google.com/drive/1c5ORIlqs3YMWosDSMgs6_ZHb5eiANS1c?usp=sharing)

Making some changes to it with the only intention of trying ideas and learning.

Notice that I am assuming you have the relevant API_KEYs as environmental variables.

In [11]:
from bubls.utils.agents.booking_tools import (
    get_booking_state_tool,
    update_booking_tool,
    create_booking_tool,
    confirm_booking_tool
)
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

Things to know:
- We will define an Agent that manages bookings for a restaurant.
    - We will use a `Booking` Pydantic class to track and represent a booking.
    - We will have the next methods:
        - `get_booking_state`: Use booking ID to retrieve the state of a booking. With return_direct=True, no LLM is used to generate the output
        - `update_booking`: Update the booking by modifying a specific property. With return_direct=False, LLM is used to generate the output
        - `create_booking`: Initialize a booking and requesting more info. With return_direct=True, no LLM is used to generate the output
        - `confirm_booking`: Complete the booking by making sure all information has been provided With return_direct=True, no LLM is used to generate the output

## Booking Tools

In [2]:
tools = [
    get_booking_state_tool,
    update_booking_tool,
    create_booking_tool,
    confirm_booking_tool,
]

In [3]:
prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"You are now connected to the booking system and helping a user with making a booking. "
            "Only enter details that the user has explicitly provided."
            "Do not make up any details."
        ),
    )
]

In [4]:
worker = FunctionCallingAgentWorker(
    tools=tools,
    llm=OpenAI(model='gpt-3.5-turbo'),
    prefix_messages=prefix_messages,
    max_function_calls=10,
    allow_parallel_tool_calls=False,
    verbose=True,
)
agent = AgentRunner(worker)

In [5]:
response = agent.chat("Hello! I would like to make a booking.")

Added user message to memory: Hello! I would like to make a booking.


=== Calling Function ===
Calling function: create_booking with args: {}
=== Function Output ===
Booking created with id EMZQLL, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [6]:
response = agent.chat("Sure! My name is bubl-ai, and my email is contact.bubl.ai@gmail.com")

Added user message to memory: Sure! My name is bubl-ai, and my email is contact.bubl.ai@gmail.com


=== Calling Function ===
Calling function: update_booking with args: {"property": "name", "user_id": "EMZQLL", "value": "bubl-ai"}
=== Function Output ===
Booking ID EMZQLL updated with name = bubl-ai
=== Calling Function ===
Calling function: update_booking with args: {"property": "email", "user_id": "EMZQLL", "value": "contact.bubl.ai@gmail.com"}
=== Function Output ===
Booking ID EMZQLL updated with email = contact.bubl.ai@gmail.com
=== LLM Response ===
Thank you for providing your name and email. Could you please provide your phone number, preferred date, and time for the booking?


In [7]:
response = agent.chat("My phone number is 111-222-3456, reservation for this sunday at 10 am")

Added user message to memory: My phone number is 111-222-3456, reservation for this sunday at 10 am


=== Calling Function ===
Calling function: update_booking with args: {"property": "phone", "user_id": "EMZQLL", "value": "111-222-3456"}
=== Function Output ===
Booking ID EMZQLL updated with phone = 111-222-3456
=== Calling Function ===
Calling function: update_booking with args: {"property": "date", "user_id": "EMZQLL", "value": "this sunday"}
=== Function Output ===
Booking ID EMZQLL updated with date = this sunday
=== Calling Function ===
Calling function: update_booking with args: {"property": "time", "user_id": "EMZQLL", "value": "10 am"}
=== Function Output ===
Booking ID EMZQLL updated with time = 10 am
=== LLM Response ===
Thank you for providing all the details. Your booking is almost complete. Would you like to confirm the booking for this Sunday at 10 am?


In [8]:
response = agent.chat("yes")

Added user message to memory: yes
=== Calling Function ===
Calling function: confirm_booking with args: {"user_id": "EMZQLL"}
=== Function Output ===
Booking for Name: bubl-ai and ID: EMZQLL is confirmed!


In [9]:
response = agent.chat("provide the booking details")

Added user message to memory: provide the booking details
=== Calling Function ===
Calling function: get_booking_state with args: {"user_id": "EMZQLL"}
=== Function Output ===
{'name': 'bubl-ai', 'email': 'contact.bubl.ai@gmail.com', 'phone': '111-222-3456', 'date': 'this sunday', 'time': '10 am'}


In [17]:
response = agent.chat("Write as list all user the IDs and the name is associated with each.")

Added user message to memory: Write as list all user the IDs and the name is associated with each.
=== LLM Response ===
1. Booking ID: EMZQLL, Name: bubl-ai
